In [111]:
import pandas as pd
from collections import Counter, defaultdict
from tqdm import tqdm_notebook as tqdmn

pd.set_option('display.max_rows', 1200)

In [112]:

df = pd.read_csv('~/hdd/proj/XKEc/results/NELL186/TransE/1526711822_1904171211/pra_explain/results/g_2negrate_bern__pra/concept:stadiumlocatedincity/train.tsv', sep='\t', names=['triple', 'label', 'paths'])
# df = pd.read_csv('~/hdd/proj/XKEc/results/FB13/TransE/1527033688_1904171027/pra_explain/results/g_2negrate_bern__pra/religion/train.tsv', sep='\t', names=['triple', 'label', 'paths'])
df['paths'] = df['paths'].fillna('')

In [113]:
df.head()

,triple,label,paths
0,"concept:building:flamingo_las_vegas001,concept...",1,-concept:locationlocatedwithinlocation-concept...
1,"concept:building:flamingo_las_vegas001,concept...",-1,-concept:stadiumlocatedincity-concept:cityloca...
2,"concept:city:osaka,concept:city:los_angeles",-1,-concept:citylocatedincountry-_concept:automob...
3,concept:stadiumoreventvenue:american_airlines_...,1,-_concept:teamhomestadium-_concept:athleteleds...
4,"concept:wine:merlot,concept:city:dallas",-1,-_concept:beveragemadefrombeverage-concept:agr...


In [114]:
train_set = pd.read_csv('~/proj/XKEc/benchmarks/NELL186/train2id.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])
valid_set = pd.read_csv('~/proj/XKEc/benchmarks/NELL186/valid2id.txt', sep=' ', skiprows=1, names=['e1', 'e2', 'rel'])

In [115]:
graph = train_set.append(valid_set, ignore_index=True)

In [116]:
graph[(graph['e1'] == 35371) | (graph['e2'] == 35371)]

,e1,e2,rel


In [117]:
logit = pd.read_csv('~/hdd/proj/XKEc/results/NELL186/TransE/1526711822_1904171211/pra_explain/results_explained/g_2negrate_bern__pra/global_logit/concept:stadiumlocatedincity', sep='\t')

FileNotFoundError: File b'/home/andrey/hdd/proj/XKEc/results/NELL186/TransE/1526711822_1904171211/pra_explain/results_explained/g_2negrate_bern__pra/global_logit/concept:stadiumlocatedincity' does not exist

In [103]:
logit

,weight,feature
0,1.811185,-concept:actorstarredinmovie-_concept:director...
1,1.455399,-concept:actorstarredinmovie-_concept:actorsta...
2,0.308328,-concept:directordirectedmovie-
3,0.308328,-concept:directordirectedmovie-_concept:actors...
4,0.308328,-concept:directordirectedmovie-_concept:direct...
5,0.072506,-concept:directordirectedmovie-_concept:actors...
6,-0.022873,-concept:personhasresidenceinstateorprovince-c...
7,-0.022873,-concept:personhasresidenceinstateorprovince-c...


In [104]:
triple_names = df['triple'].tolist()
labels = df['label'].tolist()
path_list = df['paths'].tolist()

In [105]:
path_list[0][:-4].split(',1.0 -#- ')

['']

In [106]:
path_triples = defaultdict(list)
path_count = dict()

for i in tqdmn(range(len(triple_names))):

    paths = path_list[i][:-4].split(',1.0 -#- ')
    for j in range(len(paths)):
        try:
            current = path_triples[paths[j]]
        except:
            current = []
        
        try:
            count = path_count[paths[j]]
        except:
            count = 0
        
        path_triples[paths[j]] = current + [triple_names[i]]
        path_count[paths[j]] = count + 1

In [107]:
len(path_count.keys())

9

In [108]:
path_count

{'': 188,
 '-concept:actorstarredinmovie-_concept:directordirectedmovie-concept:directordirectedmovie-': 18,
 '-concept:actorstarredinmovie-_concept:actorstarredinmovie-concept:actorstarredinmovie-': 13,
 '-concept:directordirectedmovie-': 2,
 '-concept:directordirectedmovie-_concept:directordirectedmovie-concept:directordirectedmovie-': 2,
 '-concept:directordirectedmovie-_concept:actorstarredinmovie-concept:directordirectedmovie-': 2,
 '-concept:directordirectedmovie-_concept:actorstarredinmovie-concept:actorstarredinmovie-': 1,
 '-concept:personhasresidenceinstateorprovince-concept:statelocatedingeopoliticallocation-_concept:weaponmadeincountry-concept:weaponmadeincountry-': 1,
 '-concept:personhasresidenceinstateorprovince-concept:statelocatedincountry-_concept:weaponmadeincountry-concept:weaponmadeincountry-': 1}

In [109]:
logit['count'] = logit['feature'].map(path_count)

In [110]:
logit

,weight,feature,count
0,1.811185,-concept:actorstarredinmovie-_concept:director...,18
1,1.455399,-concept:actorstarredinmovie-_concept:actorsta...,13
2,0.308328,-concept:directordirectedmovie-,2
3,0.308328,-concept:directordirectedmovie-_concept:actors...,2
4,0.308328,-concept:directordirectedmovie-_concept:direct...,2
5,0.072506,-concept:directordirectedmovie-_concept:actors...,1
6,-0.022873,-concept:personhasresidenceinstateorprovince-c...,1
7,-0.022873,-concept:personhasresidenceinstateorprovince-c...,1


In [92]:
logit[logit['weight'] != 0]['count'].mean()

100.61036036036036